In [25]:
!v4l2-ctl -d /dev/video0 --list-formats-ext

ioctl: VIDIOC_ENUM_FMT
	Type: Video Capture

	[0]: 'MJPG' (Motion-JPEG, compressed)
		Size: Discrete 4096x1536
			Interval: Discrete 0.033s (30.000 fps)
			Interval: Discrete 0.067s (15.000 fps)
			Interval: Discrete 0.100s (10.000 fps)
		Size: Discrete 3840x1080
			Interval: Discrete 0.033s (30.000 fps)
			Interval: Discrete 0.067s (15.000 fps)
			Interval: Discrete 0.100s (10.000 fps)
		Size: Discrete 1600x600
			Interval: Discrete 0.033s (30.000 fps)
			Interval: Discrete 0.067s (15.000 fps)
			Interval: Discrete 0.100s (10.000 fps)
		Size: Discrete 2560x720
			Interval: Discrete 0.033s (30.000 fps)
			Interval: Discrete 0.067s (15.000 fps)
			Interval: Discrete 0.100s (10.000 fps)
		Size: Discrete 1280x480
			Interval: Discrete 0.033s (30.000 fps)
			Interval: Discrete 0.067s (15.000 fps)
			Interval: Discrete 0.100s (10.000 fps)
			Interval: Discrete 0.033s (30.000 fps)
			Interval: Discrete 0.067s (15.000 fps)
			Interval: Discrete 0.100s (10.000 fps)
		Size: Discrete 640x240
			

In [26]:
import cv2

In [27]:
cap = cv2.VideoCapture(0)
tm = cv2.TickMeter()

In [28]:
if not cap.isOpened():
    print("Camera not found")
    exit()

cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
# cap.set(cv2.CAP_PROP_TILT, 36000)
# cap.set(cv2.CAP_PROP_ZOOM, 100)

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
tilt = cap.get(cv2.CAP_PROP_TILT)

print(f"{int(width)}x{int(height)}")
print(f"tilt: {tilt}")
print(cap.get(cv2.CAP_PROP_ZOOM))

640x240
tilt: 0.0
100.0


In [29]:
while True:
    tm.start()

    ret, frame = cap.read()

    if not ret:
        print("Do not read frame")
        break

    h, w  = frame.shape[:2]

    # 가로 분할
    half_width = w // 2

    right_camera = frame[0:h, 0:half_width]
    left_camera = frame[0:h, half_width:w]

    # 필터
    # left_camera_gray = cv2.cvtColor(left_camera, cv2.COLOR_BGR2GRAY)
    # left_camera = cv2.cvtColor(left_camera_gray, cv2.COLOR_GRAY2BGR)

    # # 단일 출력
    # combined_frame = right_camera

    # 합치기
    combined_frame = cv2.hconcat([left_camera, right_camera])

    tm.stop()
    fps = tm.getFPS()

    cv2.putText(combined_frame, f"FPS: {round(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Camera", combined_frame)
    
    if cv2.waitKey(1) & 0xff == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()